# Build YAC

We create a work directory and setup the environement.

`source activate_levante_env` must be executed at the beginning of every script/notebook to setup the environment variables such that all dependencies can be found.

In [ ]:
mkdir -p YAC
source activate_levante_env

## Environment
We setup a dedicated python environment here for the examples using python. This is automatically enabled when you `source activate_levante_env`.

In [ ]:
python -m venv ${workdir}/.venv
# no need to activate the env. PATH variable is already set in env
pip install --upgrade pip
pip install wheel cython pyyaml isodate numpy matplotlib mpi4py netcdf4 xarray cartopy

## YAXT

In [ ]:
pushd YAC
git clone --depth=1 -b "release-0.11.1" https://gitlab.dkrz.de/dkrz-sw/yaxt.git "yaxt"
popd

Configure, build and install YAXT:

In [ ]:
pushd YAC/yaxt
yaxt_install_dir=`pwd -P`/install
mkdir -p build
cd build
../configure -q --enable-silent-rules --prefix=${yaxt_install_dir}
make -j10
make install
popd

Run the test suite of YAXT (optional):

In [ ]:
pushd YAC/yaxt/build
make check
popd

## YAC

In [ ]:
pushd YAC
git clone --depth=1 -b "release-3.2.0" https://gitlab.dkrz.de/dkrz-sw/yac.git "yac"
popd

Configure, build and Install YAC:

In [ ]:
pushd YAC/yac
yac_install_dir=`pwd -P`/install
mkdir -p build
cd build
../configure --prefix=${yac_install_dir} \
  CC=mpicc FC=mpif90 CFLAGS="-O0 -g" FCFLAGS="-O0 -g" \
    --with-yaxt-root="${yaxt_install_dir}" \
    MKL_CFLAGS="`pkg-config --cflags mkl-static-lp64-seq`" \
    MKL_CLIBS="`pkg-config --libs mkl-static-lp64-seq`" \
    -q --enable-silent-rules --enable-python-bindings
make -j10
make install
popd

Install YAC also in python venv:

In [ ]:
pip install YAC/yac/build/python

Run the test suite of YAC (optional):

In [ ]:
pushd YAC/yac/build
make check
popd

# Make

Build the toy atmosphere and ocean model:

In [ ]:
make

Run the models separately:

In [ ]:
mpirun -n 4 toy_atm.x

In [ ]:
mpirun -n 5 toy_ocn.x

Run them simultaneously:

In [ ]:
mpirun -n 4 toy_atm.x : -n 5 toy_ocn.x

Question: Why does it fail?